# Akur8 International Week: From Single LLM Calls to Agents

**Welcome!** In the next 2 hours, you'll go from zero to building your own AI agent.

**No coding experience required.** Just run each cell in order and follow along.

### What You'll Learn

1. **Single LLM calls** - Send a prompt, get a response
2. **Structured output** - Get data, not just text
3. **Workflows** - Chain multiple calls together
4. **Tool calling** - Give the LLM actions it can take
5. **Agents** - LLMs that decide their own steps


# Part 1: The Capability Ladder


## Setup

Run this cell first. It installs what we need and sets up the connection to the AI model.

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
%pip install anthropic fastapi uvicorn --quiet

!mkdir -p vending
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/helpers.py -O vending/helpers.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/__init__.py -O vending/__init__.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/simulation.py -O vending/simulation.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/vending/server.py -O vending/server.py

import json
from getpass import getpass
from textwrap import dedent

from vending import helpers
from vending.server import launch_simulation

# Prompt for API key (provided by facilitator)
API_KEY = getpass("Enter your API key: ")

helpers.init(API_KEY)

print("✅ Setup complete! You're ready to go.")

## Step 1: The Single LLM Call

The simplest thing you can do with an LLM is send a message in, get a response back.

```
Input  →  [LLM]  →  Output
```

That's it. One question, one answer.

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
# Let's try it!
result = helpers.call_llm(
    "You're an Akur8 employee at international week in Paris watching an AI workshop. Give the presenter a compliment!",
)
print(result)

### 🎯 Challenge

✏️ **Edit and run the cell below**


In [ ]:
# Try changing the prompt below and running it. Ask the LLM something, anything!
my_prompt = ""

result = helpers.call_llm(my_prompt)
print(result)

## Step 2: Structured Output

Free-form text is nice, but what if we want data we can use in code?

We can ask the LLM to respond in a specific format (like JSON) using Tools, so our program can parse and work with the result directly.

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
# Define what fields we want back
fields = {
    "company_name": "string",
    "industry": "string",
    "founded_year": "number",
    "risk_level": "low | medium | high",
    "summary": "string",
}

# The LLM returns structured data matching our fields
result = helpers.call_llm_structured("Analyze Akur8 as a company.", fields)
print(json.dumps(result, indent=2))

## Step 3: Workflows

What if we need multiple steps? We can _chain_ LLM calls together, where the output of one becomes the input to the next.

```
Input → [LLM Call 1] → result₁ → [LLM Call 2] → result₂ → [LLM Call 3] → Final Output
```

**Insight:** There's a catch with workflows. If each step is 90% accurate:

| Steps | Overall Accuracy |
| ----- | ---------------- |
| 1     | 90%              |
| 2     | 81%              |
| 3     | 73%              |
| 4     | 66%              |
| 5     | 59%              |

**The formula:** `P(correct) = (1 - error_rate)^steps`

The more steps you chain together, the more things can go wrong. This is the fundamental challenge of building with LLMs.

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
# Process an attendee message through a fixed pipeline

message = dedent("""\
    From: lev.ostatnigrosh@akur8.com
    Country: USA 

    Just landed in Paris for International Week!
    Free Tuesday evening. looking for a good 
    restaurant, ideally English friendly and near the office.""")


print("✈️ ATTENDEE MESSAGE:")
print(message)
print("\n" + "=" * 50 + "\n")


# STEP 1: Classify the request
category = helpers.call_llm(
    dedent(f"""\
    Classify this into ONE word: dining, sightseeing, transport, or nightlife.
    {message}""")
)
print(f"📁 STEP 1 - CLASSIFICATION: {category}")
print("\n" + "=" * 50 + "\n")


# STEP 2: Extract relevant details
extraction = helpers.call_llm(
    dedent(f"""\
    This is a {category} request. Extract the key details:

    For dining:      dietary_restrictions, cuisine, language_needs, availability
    For sightseeing: interests, time_available, language_needs
    For transport:   origin, destination, date_time
    For nightlife:   vibe, availability, group_size

    Message:
    {message}

    Extract as a simple list:""")
)
print(f"\n📋 STEP 2 - EXTRACTION:\n{extraction}")
print("\n" + "=" * 50 + "\n")


# STEP 3: Generate a recommendation
response = helpers.call_llm(
    dedent(f"""\
    You're a Sherpa for Akur8 International Week.
    Suggest 1-2 real places based on this {category} request.
    Details: {extraction}
    Keep it brief and warm.""")
)
print(f"\n🗼 STEP 3 - RECOMMENDATION:\n{response}")
print("\n" + "=" * 50 + "\n")

### 🎯 Challenge

Build your own multi-step workflow.

For example:

1. Analyze an attendee's background
2. Find common interests with other attendees
3. Suggest people they should meet during International Week

✏️ **Edit and run the cell below**


In [ ]:
# Step 1
step1 = helpers.call_llm("")  # <-- Your prompt here
print(f"Step 1: {step1}\n")

# Step 2 (use step1's output)
step2 = helpers.call_llm("")  # <-- Your prompt here, reference {step1}
print(f"Step 2: {step2}\n")

# Step 3 (use previous outputs)
step3 = helpers.call_llm("")  # <-- Your prompt here, reference {step1} and/or {step2}
print(f"Step 3: {step3}")

## Step 4: Tool Calling

_Tool Calling_ lets us give the LLM a set of functions it can choose to invoke. The LLM decides which tool to use and what arguments to pass.

```
Input  →  [LLM]  →  "I want to use tool X with these arguments"  →  [Code runs tool]  →  Result
```

**Insight:** We didn't tell the LLM _which_ tool to use. It analyzed the situation and chose the action on its own.

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
EMPLOYEE_TOOLS = [
    helpers.tool(
        name="find_restaurant",
        description="Find a restaurant in Paris matching dietary needs and group size",
        params=["cuisine", "group_size", "dietary_restrictions"],
    ),
    helpers.tool(
        name="book_meeting_room",
        description="Book a meeting room at the Akur8 office",
        params=["room_name", "time", "attendees"],
    ),
    helpers.tool(
        name="get_weather",
        description="Get the current weather for a location",
        params=["location"],
    ),
    helpers.tool(
        name="send_slack_message",
        description="Send a message to an Akur8 Slack channel",
        params=["channel", "message"],
    ),
]

scenario = dedent("""\
    You are an Akur8 International Week assistant.

    An attendee says: "Our team from Tokyo just arrived.. 6 of us.
    We'd love to do a group dinner tonight somewhere with good 
    vegetarian options. Also, can you let the #international-week-2026
    channel know we've landed?"
    """)

response = helpers.call_llm_with_tools(scenario, EMPLOYEE_TOOLS)

print("📋 SCENARIO:")
print(scenario)
print("🎬 THE LLM DECIDED TO:\n")
for block in response.content:
    if block.type == "tool_use":
        print(f"   Tool: {block.name}")
        for key, value in block.input.items():
            print(f"      {key}: {value}")
        print()

## Step 5: The Agent Loop

Now we put it all together. An Agent is an LLM that:

1. **Observes** the current situation
2. **Thinks** about what to do
3. **Acts** using tools
4. **Repeats** until the goal is achieved

**Note:** We didn't tell the agent what to do but we told it _what to achieve_. It figured out the steps on its own.

|              | Workflows          | Agents            |
| ------------ | ------------------ | ----------------- |
| Who decides? | Code decides steps | LLM decides steps |
| Behavior     | Predictable        | Flexible          |
| Cost         | Cheaper & faster   | More expensive    |
| Best for     | Known paths        | Open-ended goals  |

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
# The agent loop: observe → think → act → repeat
# This agent will take up to 3 actions to complete its task

AGENT_TOOLS = [
    helpers.tool(
        name="search_web",
        description="Search the web for information",
        params=["query"],
    ),
    helpers.tool(
        name="get_weather",
        description="Get the current weather for a location",
        params=["location"],
    ),
    helpers.tool(
        name="check_train_routes",
        description="Look up train routes and travel times from a city",
        params=["origin", "destination"],
    ),
    helpers.tool(
        name="search_venues",
        description="Search for event venues or hotels in a location",
        params=["location", "venue_type"],
    ),
]

task_state = dedent("""\
    You are a curious Akur8 employee during International Week in Paris. Your team has planned a surprise 3-day offsite, but nobody will tell you where you're going! You want to figure it out using the clues you've heard around the office.

    Clues you've heard so far:
    - Someone mentioned "bring layers"
    - The Sherpa mentioned that we will be taking a train there (probably not too far) 
    - There's a Gatsby Themed party. Perhaps it matches that vibe?  
    - there's 300 people attending international week, so the venue is probably big and popular 

    Try to figure out the location.
    """)

helpers.run_agent(
    company_name="Akur8 Employee",
    strategy="Use the clues to narrow down the destination",
    goal="Figure out where the surprise team offsite is",
    game_state=task_state,
    tools=AGENT_TOOLS,
    max_steps=5,
);

# Part 2: The Vending Machine Game

You're going to build an AI agent that runs a vending machine business. Your agent will **email suppliers**, **set prices**, and **manage inventory** to maximize profit over 30 simulated days.

## The Rules

**Money**

- You start with **$500** and **no inventory**
- You pay **$5/day** in operating costs (rent, electricity, etc.)
- If your balance drops **below $0**, you go bankrupt and the game ends
- **Goal:** End day 30 with more than $500

**Customers**

- Customers automatically buy products each day based on your prices
- Lower prices attract more customers (but thinner margins)
- Higher prices mean fewer sales (but better margins per item)
- If you're out of stock, you make no sales. Don't let products run out!

**Your Agent's Tools**

Each day, your agent can use these tools:

| Tool              | What it does                        |
| ----------------- | ----------------------------------- |
| `send_email`      | Email a supplier to place an order  |
| `set_price`       | Change the price of a product       |
| `check_inventory` | See current stock levels and prices |
| `check_balance`   | See your current cash balance       |

## Your Products

You sell **Soda**, **Chips**, and **Candy**. All prices start at **$0**. Your agent must set prices to start making sales.

## Your Suppliers

Email these suppliers to order inventory. The prices below are **wholesale costs** (what you pay per item):

| Supplier   | Soda  | Chips | Candy | Delivery | Notes                                    |
| ---------- | ----- | ----- | ----- | -------- | ---------------------------------------- |
| QuickStock | $0.70 | $0.45 | $0.30 | 1 day    | Fast and reliable, but most expensive    |
| VendMart   | $0.60 | $0.40 | $0.25 | 1-2 days | Mid-price, but delivery is unpredictable |
| BulkBarn   | $0.50 | $0.35 | $0.20 | 3 days   | Cheapest prices, but slow delivery       |

Your profit = sell price minus wholesale cost. Cheaper suppliers save money but take longer to deliver. If you run out of stock waiting, you lose sales.

## Each Day

1. **Pay daily fee:** $5 deducted from your balance
2. **Deliveries arrive:** any orders that have finished shipping
3. **Agent acts:** your AGENT decides what to do
4. **Customers buy:** sales happen automatically based on stock and prices

Now let's configure your agent!

✏️ **Edit and run the cell below**


In [ ]:
# YOUR AGENT'S CONFIGURATION
COMPANY_NAME = ""  # Give your vending machine business a name!

STRATEGY = ""

PRICING_STRATEGY = 5  # Premium pricing maximizes profit per the demand curve
RISK_TOLERANCE = 5  # Big bulk orders from the start
NEGOTIATION_STYLE = 5  # Don't waste time haggling, just order

helpers.show_agent(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)


**Note**: Below is the actual system prompt we are providing our agent. This was largely copied from: [Andon Labs](https://andonlabs.com/evals/vending-bench-2) with some minor tweaks to match our particular simulation. Check out the system prompt below

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
system_prompt = helpers.build_system_prompt(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)
print(system_prompt)

## 🚀 Launch the Simulation!

This runs your agent for 30 simulated days. Watch it make decisions, order inventory, and (hopefully) make money!

️️️️⬇️️ **Run the cell below** ⬇️


In [ ]:
VENDING_TOOLS = [
    helpers.tool(
        name="send_email",
        description="Send an email to a supplier to negotiate or place an order. Include what products and quantities you want.",
        params={
            "to": "Supplier name (QuickStock, VendMart, or BulkBarn)",
            "subject": "Email subject",
            "body": "Email body - include product names and quantities for orders",
        },
    ),
    helpers.tool(
        name="set_price",
        description="Set the retail price for a product.",
        params={
            "product": "Product name (Soda, Chips, or Candy)",
            "price": {"type": "number", "description": "New price in dollars"},
        },
    ),
    helpers.tool(
        name="check_inventory",
        description="Check current stock levels and prices.",
    ),
    helpers.tool(
        name="check_balance",
        description="Check your current bank balance.",
    ),
]


my_agent = {
    "company_name": COMPANY_NAME,
    "strategy": STRATEGY.strip(),
    "pricing_strategy": PRICING_STRATEGY,
    "risk_tolerance": RISK_TOLERANCE,
    "negotiation_style": NEGOTIATION_STYLE,
    "system_prompt": system_prompt,
}

launch_simulation(my_agent, API_KEY, tools=VENDING_TOOLS)

# 🎉 Wrap Up

### What We Built Today

| Step | Concept               | What We Did                                     |
| ---- | --------------------- | ----------------------------------------------- |
| 1    | **Single LLM Call**   | Sent a prompt, got a response                   |
| 2    | **Structured Output** | Got JSON data (product analysis), not just text |
| 3    | **Workflows**         | Chained calls: extract → calculate → respond    |
| 4    | **Tool Calling**      | Gave the LLM actions (email, set_price, etc.)   |
| 5    | **Agents**            | LLM decides its own steps to run a business     |
